# Clustering analysis

In [ ]:
import os, sys
import warnings

import pandas as pd
import numpy as np

from scipy.stats import zscore
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering 

from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

In [ ]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
warnings.filterwarnings('ignore')

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext watermark
%watermark -a 'Fernando Pozo' -u -n -t -z -g -p pandas,numpy

!pwd

In [ ]:
def process_logratios(df):
    df = df[df.columns[4:]].set_index('peptide')
    df['std'] = df.std(axis=1)
#     df = df.sort_values('std',ascending = False).drop('std', axis=1).head(int(df.shape[0]/4))
    df = df.sort_values('std',ascending = False).drop('std', axis=1)
    df_scaled = pd.DataFrame(zscore(df.values, axis=0), index=df.index, columns=df.columns)
    return df_scaled


def time_series_kmeans_plot(df:list, n_clusters:int, title:str, seed:int=123):
    '''
    https://tslearn.readthedocs.io/en/stable/auto_examples/clustering/plot_kmeans.html#sphx-glr-auto-examples-clustering-plot-kmeans-py
    '''
    data = df.values
    labels = list(df.columns)
    clusters = n_clusters
    
    X_train = TimeSeriesScalerMeanVariance().fit_transform(data)
#     X_train = TimeSeriesResampler(sz=3).fit_transform(X_train)
    sz = X_train.shape[1]
    km = TimeSeriesKMeans(n_clusters=clusters, metric="dtw", max_iter=10, verbose=True, random_state=seed)
    y_pred = km.fit_predict(X_train)

    fig, axes = plt.subplots(1, 1, sharex=True, sharey=True, figsize=(20,8), 
    #                              dpi=1200
                                )
    fig.suptitle(title, fontsize=16, y=0.98)
    for yi in range(clusters):
        plt.subplot(3, clusters, yi + 1)
        for xx in X_train[y_pred == yi]:
            plt.plot(xx.ravel(), "k-", alpha=.02)
            plt.xticks(np.linspace(0, sz, sz+1))
        plt.plot(km.cluster_centers_[yi].ravel(), "r-")
    #     plt.xlim(0, sz)
        plt.ylim(-3, 3)
        plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
                 transform=plt.gca().transAxes)
    fig.text(0.5, 0.6, 'Timepoints', ha='center', fontsize=14)
    fig.text(0.09, 0.77, 'Normalized log ratio', va='center', rotation='vertical', fontsize=14)
    return fig, y_pred

# 1. Data Loading

In [ ]:
df_wt = pd.read_excel('../data/external/files/September_2018_Julio_WT_TC_1-48_rep_replicate3_20180921s_with_LP.xlsx')
df_ko = pd.read_excel('../data/external/files/Julio_KO_PO4_1-30_20170921_Final_PEPTIDE.xlsx')

df_wt_log = df_wt[['ptm', 'proteinName', 'proteinDescription', 'FTestQValue', 'peptide',
       'log2ratio_0-0', 'log2ratio_1c', 'log2ratio_2c', 'log2ratio_5c', 
       'log2ratio_10c', 'log2ratio_20c', 'log2ratio_40c', 'log2ratio_80c']]
df_wt_log = df_wt_log[df_wt_log['FTestQValue']>0.05]

df_ko_log = df_ko[['ptm', 'proteinName', 'proteinDescription', 'FTestQValue', 'peptide',
       'log2ratio_KO-0', 'log2ratio_KOc-1', 'log2ratio_KOc-2',
       'log2ratio_KOc-5', 'log2ratio_KOc-10', 'log2ratio_KO-20']]
df_ko_log = df_ko_log[df_ko_log['FTestQValue']>0.05]

df_wt_log_data = process_logratios(df_wt_log)
df_wt_log_data.columns = [0, 1, 2, 5, 10, 20, 40, 80]

df_ko_log_data = process_logratios(df_ko_log)
df_ko_log_data.columns = [0, 1, 2, 5, 10, 20]

# 2. Data Exploration

Exploring the best set of clusters with dendrogram with ward (minimizing variances between them)

In [ ]:
data = df_wt_log_data
plt.figure(figsize=(20,6))
dendrogram = sch.dendrogram(sch.linkage(data, method  = "ward"))
plt.title('Wild-type peptides dendrogram')
plt.xlabel(f'Set of {data.shape[0]} peptides')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
data = df_ko_log_data
plt.figure(figsize=(20,6))
sch.dendrogram(sch.linkage(data, method  = "ward"))
plt.title('Knock-out peptides dendrogram')
plt.xlabel(f'Set of {data.shape[0]} peptides', fontsize=16)
plt.ylabel('Euclidean distances', fontsize=16)
plt.show()

In [ ]:
fig_wt, y_pred_wt = time_series_kmeans_plot(df_wt_log_data, 3, 'A')
df_wt_log_data['cluster_kmeans'] = y_pred_wt + 1

In [ ]:
fig_ko, y_pred_ko = time_series_kmeans_plot(df_ko_log_data, 3, 'B')
df_ko_log_data['cluster_kmeans'] = y_pred_ko + 1

In [ ]:
fig_wt.savefig('../reports/figures/figure4A.png', bbox_inches='tight')
fig_ko.savefig('../reports/figures/figure4B.png', bbox_inches='tight')

In [ ]:
df_wt_log_data['cluster'] = AgglomerativeClustering(n_clusters = 6, affinity = 'euclidean', linkage ='ward').fit_predict(df_wt_log_data) + 1
df_ko_log_data['cluster'] = AgglomerativeClustering(n_clusters = 6, affinity = 'euclidean', linkage ='ward').fit_predict(df_ko_log_data) + 1

In [ ]:
def plot_clusters(df, title):
    rows = 3
    cols = 2
    timepoints = df.drop('cluster', axis=1).columns
    fig, axes = plt.subplots(rows, cols, sharex=True, sharey=True, figsize=(12,8), 
                             dpi=1200
                            )
    fig.suptitle(title, fontsize=20, y=1.05)

    for n, ax in enumerate(axes.flat):
        n=n+1
        data = df[df['cluster'] == n].drop('cluster', axis=1)
        for i in range(0, data.shape[0]):
            ax.plot(timepoints, data.iloc[i].values, marker='', linewidth=0.5, alpha=0.4, color='orange')
            ax.plot(timepoints, data.describe().iloc[1].values, marker='.', markersize=12, 
                    linestyle='solid', linewidth=0.4, alpha=0.4, color='gray')
            ax.set_ylim([-5, 5])
            ax.set_title(f'Cluster {n}')
    plt.tight_layout()

In [ ]:
plot_clusters(df_wt_log_data, 'Wild-type peptides cluster analysis')

In [ ]:
plot_clusters(df_ko_log_data, 'Knock-out peptides cluster analysis')

# Project contribution <a class="anchor" id="project-contribution"></a>

**Author information**:
Fernando Pozo ([@fpozoc](https://gitlab.com/fpozoc))

You can find the data driven project jupyter notebook template [here](https://gitlab.com/fpozoc/data-driven-project-template/-/blob/master/notebooks/1.0-nb_template.ipynb).